In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# load csv files
train_scores = pd.read_csv('/kaggle/input/trends-assessment-prediction/train_scores.csv')

# replace nan with mean of columns
#train_scores.fillna(train_scores.mean(), inplace=True)

In [2]:
predictions_train = pd.read_csv('/kaggle/input/rapids-fnc-ic/fnc_ic_train_pred')
predictions_test = pd.read_csv('/kaggle/input/rapids-fnc-ic/fnc_ic_test_pred')
predictions_train.columns = ['unnamed', 'Id', 'age', 'domain1_var1', 'domain1_var2','domain2_var1','domain2_var2']
predictions_test.columns = ['unnamed', 'Id', 'age', 'domain1_var1', 'domain1_var2','domain2_var1','domain2_var2']


In [3]:
res_train = pd.read_csv('/kaggle/input/res350/res_preds_traintime3.csv')
res_test = pd.read_csv('/kaggle/input/res350/res_preds_testtime3.csv')

In [4]:
def X_combine(col):
    return pd.DataFrame([res_train[col], predictions_train[col]]).transpose() 
def X_combine_test(col):
    return pd.DataFrame([res_test[col], predictions_test[col]]).transpose() 

In [5]:
# from sklearn.model_selection import KFold
# from sklearn.svm import SVR
# from sklearn.model_selection import GridSearchCV

# cv = KFold(n_splits=7, shuffle=True, random_state=0)
# svr = SVR(cache_size=3000.0)
# param_svr = { 'epsilon': [0.0001, 0.001, 0.1,0.3, 0.6, 1.0, 1.5],'C':[0.1, 1,  5,10,  50, 75, 100, 1000]}
# gs = GridSearchCV(svr, param_svr, cv=cv, n_jobs=-1, verbose=5, scoring='neg_mean_absolute_error')

# features = ['age', 'domain1_var1', 'domain1_var2','domain2_var1','domain2_var2']
# best_models = {}

# for target in features:
#     print(target)
#     i = 0
#     gs.fit(X_combine(target), train_scores[target])
    
#     best_models[target] = gs.best_estimator_, gs.best_score_
    

In [6]:
# best_models

In [7]:
# {'age': (SVR(C=5, cache_size=3000.0, coef0=0.0, degree=3, epsilon=1.5, gamma='scale',
#       kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
#   -7.1616849988982665),
#  'domain1_var1': (SVR(C=1, cache_size=3000.0, coef0=0.0, degree=3, epsilon=0.6, gamma='scale',
#       kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
#   -7.3780089885075295),
#  'domain1_var2': (SVR(C=0.1, cache_size=3000.0, coef0=0.0, degree=3, epsilon=0.3, gamma='scale',
#       kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
#   -8.305435766761942),
#  'domain2_var1': (SVR(C=1, cache_size=3000.0, coef0=0.0, degree=3, epsilon=0.0001, gamma='scale',
#       kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
#   -8.509954554433026),
#  'domain2_var2': (SVR(C=0.1, cache_size=3000.0, coef0=0.0, degree=3, epsilon=1.5, gamma='scale',
#       kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
#   -9.092468570720309)}

In [8]:
predictions_train.columns = ['unnamed', 'Id', 'age_svr', 'domain1_var1_svr', 'domain1_var2_svr','domain2_var1_svr','domain2_var2_svr']
predictions_test.columns = ['unnamed', 'Id', 'age_svr', 'domain1_var1_svr', 'domain1_var2_svr','domain2_var1_svr','domain2_var2_svr']
predictions_train.drop(columns='unnamed', inplace=True)
predictions_test.drop(columns='unnamed', inplace=True)

In [9]:
res_train.columns = ['Id', 'age_res', 'domain1_var1_res', 'domain1_var2_res','domain2_var1_res','domain2_var2_res']
res_test.columns = ['Id', 'age_res', 'domain1_var1_res', 'domain1_var2_res','domain2_var1_res','domain2_var2_res']

In [10]:
df = pd.concat([predictions_train.merge(res_train, on= 'Id'), predictions_test.merge(res_test, on= 'Id')])

In [11]:
train_scores['is_train'] = True

In [12]:
df = df.merge(train_scores, on="Id", how="left")


In [13]:
test_df = df[df["is_train"] != True].copy()
df = df[df["is_train"] == True].copy()

In [14]:
test_df.shape, df.shape

((5877, 17), (5877, 17))

In [15]:
def metric(y_true, y_pred):
    return np.mean(np.sum(np.abs(y_true - y_pred), axis=0)/np.sum(y_true, axis=0))

In [16]:
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression


NUM_FOLDS = 7
kf = KFold(n_splits=NUM_FOLDS, shuffle=True)


features = ['age_svr', 'domain1_var1_svr', 'domain1_var2_svr', 'domain2_var1_svr',
       'domain2_var2_svr','age_res', 'domain1_var1_res', 'domain1_var2_res', 'domain2_var1_res',
       'domain2_var2_res']

overal_score = 0
for target, c, e, w in [("age", 5, 1.5, 0.3), ("domain1_var1", 1, 0.6, 0.175), ("domain1_var2", 0.1, 0.3, 0.175), ("domain2_var1", 1, 0.0001,  0.175), ("domain2_var2", 0.1, 1.5, 0.175)]:    
    y_oof = np.zeros(df.shape[0])
    y_test = np.zeros((test_df.shape[0], NUM_FOLDS))
    
    for f, (train_ind, val_ind) in enumerate(kf.split(df, df)):
        train_df, val_df = df.iloc[train_ind], df.iloc[val_ind]
        train_df = train_df[train_df[target].notnull()]

        model = SVR(C=c, epsilon=e, cache_size=3000.0, verbose=True)
        model.fit(train_df[features], train_df[target])

        y_oof[val_ind] = model.predict(val_df[features])
        y_test[:, f] = model.predict(test_df[features])
        
    df["pred_{}".format(target)] = y_oof
    test_df[target] = y_test.mean(axis=1)
    score = metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_{}".format(target)].values)
    overal_score += w*score
    print(target, np.round(score, 4))
    print()
    
print("Overal score:", np.round(overal_score, 4))

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]age 0.143

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]domain1_var1 0.1515

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]domain1_var2 0.1507

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]domain2_var1 0.1801

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]domain2_var2 0.1766

Overal score: 0.1582


In [17]:
# to beat
# [LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]age 0.144

# [LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]domain1_var1 0.1439

# [LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]domain1_var2 0.1407

# [LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]domain2_var1 0.1799

# [LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]domain2_var2 0.1756

# Overal score: 0.1552

In [18]:
sub_df = pd.melt(test_df[["Id", "age", "domain1_var1", "domain1_var2", "domain2_var1", "domain2_var2"]], id_vars=["Id"], value_name="Predicted")
sub_df["Id"] = sub_df["Id"].astype("str") + "_" +  sub_df["variable"].astype("str")

sub_df = sub_df.drop("variable", axis=1).sort_values("Id")
assert sub_df.shape[0] == test_df.shape[0]*5
sub_df.head(10)

,Id,Predicted
0,10003_age,52.790430
5877,10003_domain1_var1,51.176826
11754,10003_domain1_var2,59.473237
17631,10003_domain2_var1,46.957982
23508,10003_domain2_var2,54.209794
1,10006_age,65.817629
5878,10006_domain1_var1,55.426197
11755,10006_domain1_var2,60.205732
17632,10006_domain2_var1,50.697698
23509,10006_domain2_var2,51.439406


In [19]:
sub_df.to_csv('ensemble_sub_22.csv', index=False)